In [1]:
%pip install -Uq datasets==2.12.0 qdrant-client==1.2.0 sentence-transformers==2.2.2 torch==2.0.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from datasets import load_dataset
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from typing import List
from qdrant_client.http.models import Distance, VectorParams

In [3]:
client = QdrantClient("localhost", port=6333)

In [4]:
collection_name = "extractive-question-answering"

collections = client.get_collections()
print(collections)

collections=[CollectionDescription(name='extractive-question-answering')]


In [6]:
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# load the retriever model from huggingface model hub
retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [7]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# load the reader model into a question-answering pipeline
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)
print(reader.model, reader)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [8]:
def get_relevant_plot(question: str, top_k: int) -> List[str]:
    """
    Get the relevant plot for a given question

    Args:
        question (str): What do we want to know?
        top_k (int): Top K results to return

    Returns:
        context (List[str]):
    """
    try:
        encoded_query = retriever.encode(question).tolist()  # generate embeddings for the question

        result = client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [x.payload["title"], x.payload["plot"]] for x in result
        ]  # extract title and payload from result
        return context

    except Exception as e:
        print({e})

In [9]:
def extract_answer(question: str, context: List[str]):
    """
    Extract the answer from the context for a given question

    Args:
        question (str): _description_
        context (list[str]): _description_
    """
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c[1])

        # add the context to answer dict for printing both together, we print only first 500 characters of plot
        answer["title"] = c[0]
        results.append(answer)

    # sort the result based on the score from reader model
    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        print(f"{i+1}", end=" ")
        print(
            "Answer: ",
            sorted_result[i]["answer"],
            "\n  Title: ",
            sorted_result[i]["title"],
            "\n  score: ",
            sorted_result[i]["score"],
        )


question = "In the movie 3 Idiots, what is the name of the college where the main characters Rancho, Farhan, and Raju study"
context = get_relevant_plot(question, top_k=1)
context

[['Three Idiots',
  'Farhan Qureshi (R. Madhavan), Raju Rastogi (Sharman Joshi), and Rancchoddas "Rancho" Shyamaldas Chanchad (Aamir Khan) are three engineering students who share a room in a hostel at the Imperial College of Engineering, one of the best colleges in India. While Farhan and Raju are average students from modest backgrounds, Rancho is from a rich family. Farhan wants to become a wildlife photographer, but has joined engineering college to fulfil his father\'s wish. Raju on the other hand wants to uplift his family fortunes. Rancho is a wealthy genius who studies for the sheer joy of it. However, Rancho\'s passion is for knowledge and taking apart and building machines rather than the conventional obsession of the other students with exam ranks. With his different approach Rancho incurs the wrath of dean of college, Professor Viru Sahastrabudhhe (ViruS) (Boman Irani). Rancho irritates his lecturers by giving creative and unorthodox answers, and confronts ViruS after fello

In [10]:
extract_answer(question, context)

1 Answer:  Imperial College of Engineering 
  Title:  Three Idiots 
  score:  0.9049272537231445


In [14]:
question = "In the movie 3 Idiots, who committed suicide?"
context = get_relevant_plot(question, top_k=1)
context

[['Surveillance',
  'A series of violent deaths and the disappearance of a young woman bring FBI agents Hallaway (Bill Pullman) and Anderson (Julia Ormond) to a town in rural Nebraska. They meet the three survivors of a mysterious bloodbath; the young Stephanie (Ryan Simpkins), the cocaine-addicted Bobbi (Pell James), and the foul-mouthed police officer Bennett Kent Harper. Hallaway watches the trio\'s respective interviews with Captain Billings (Michael Ironside) and officers Wright (Charlie Newmark) and Degrasso (Gill Gayle), where they tell the story of what brought them there:\nIn a warped way to pass the day, officer Bennett, Kent Harper as his partner officer Conrad (French Stewart) watches (and both hidden from view), shoots the tires of cars driving along an isolated county road, then convince the drivers their tires blew out as a result of their speeding, and threaten them afterwards. They do so to one young couple then let them go.\nA bit later, Stephanie, traveling on vacati

In [15]:
extract_answer(question, context)

1 Answer:  police officer Bennett Kent Harper 
  Title:  Surveillance 
  score:  0.000118927942821756
